In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from collections import defaultdict

In [ ]:
class Node:
    def __init__(self, node_id, x, y, initial_energy, is_cluster_head=False, is_sink=False):
        self.id = node_id
        self.x = x
        self.y = y
        self.energy = initial_energy
        self.initial_energy = initial_energy
        self.is_cluster_head = is_cluster_head
        self.is_sink = is_sink
        self.alive = True
        self.state = "ACTIVE"  # Can be "ACTIVE" or "SLEEP"
        self.neighbors = []
        self.q_values = {}  # Dictionary to store Q-values for each neighbor
        self.cluster_head = None
        self.sensor_data_cache = None
        self.min_value = float('inf')
        self.max_value = float('-inf')
        self.sleep_counter = 0
        self.packets_sent = 0
        self.packets_received = 0
        self.hop_count = float('inf')  # Distance to sink in terms of hops

    def add_neighbor(self, node):
        if node not in self.neighbors and node.id != self.id:
            self.neighbors.append(node)
            self.q_values[node.id] = 0.0  # Initialize Q-value for this neighbor

    def distance_to(self, node):
        return np.sqrt((self.x - node.x) ** 2 + (self.y - node.y) ** 2)

    def consume_energy(self, amount):
        self.energy -= amount
        if self.energy <= 0:
            self.energy = 0
            self.alive = False
            self.state = "SLEEP"
            return False
        return True

    def transmit_energy(self, distance, packet_size):
        """Calculate energy consumed in transmitting a packet over a given distance"""
        if distance > TRANSMISSION_RANGE:
            return float('inf')  # Cannot transmit beyond range
        e_tx = (E_ELEC * packet_size) + (E_AMP * packet_size * (distance ** 2))
        return e_tx

    def receive_energy(self, packet_size):
        """Calculate energy consumed in receiving a packet"""
        return E_ELEC * packet_size

    def sense_energy(self):
        """Energy consumed in sensing data"""
        return E_SENSING

    def process_energy(self):
        """Energy consumed in processing data"""
        return E_PROC

    def read_sensor_data(self):
        """Simulates reading data from a sensor"""
        # In a real implementation, this would read from actual sensors
        # Here we generate random data within a specific range
        new_data = np.random.normal(25, 5)  # Mean 25, std dev 5
        # Add some small random changes to simulate real sensor readings
        if self.sensor_data_cache is not None:
            new_data = self.sensor_data_cache + np.random.uniform(-2, 2)
        self.sensor_data_cache = new_data
        return new_data

    def should_transmit_data(self, change_threshold):
        """Determines if data should be transmitted based on change threshold"""
        if self.sensor_data_cache is None:
            return True  # First reading should be transmitted

        # Update min/max values
        if self.sensor_data_cache < self.min_value:
            self.min_value = self.sensor_data_cache
        elif self.sensor_data_cache > self.max_value:
            self.max_value = self.sensor_data_cache

        # Check if change is significant enough to transmit
        if ((self.max_value - self.sensor_data_cache) > change_threshold or
            (self.sensor_data_cache - self.min_value) > change_threshold):
            # Reset min/max after transmission decision
            self.min_value = self.sensor_data_cache
            self.max_value = self.sensor_data_cache
            return True
        return False

    def update_q_value(self, neighbor_id, reward, max_q_next, learning_rate):
        """Update Q-value for a specific neighbor using Q-learning"""
        old_q = self.q_values[neighbor_id]
        self.q_values[neighbor_id] = (1 - learning_rate) * old_q + learning_rate * (reward + DISCOUNT_FACTOR * max_q_next)

    def get_next_hop(self):
        """Select next hop based on Q-values"""
        if not self.neighbors:
            return None

        # Filter only alive neighbors
        alive_neighbors = [n for n in self.neighbors if n.alive]
        if not alive_neighbors:
            return None

        # Choose neighbor with highest Q-value
        best_neighbor = max(alive_neighbors, key=lambda n: self.q_values[n.id])
        return best_neighbor

In [ ]:
class WSN:
    def __init__(self, width, height, num_nodes, num_clusters, transmission_range,
                 initial_energy, packet_size, learning_rate):
        self.width = width
        self.height = height
        self.num_nodes = num_nodes
        self.num_clusters = num_clusters
        self.transmission_range = transmission_range
        self.initial_energy = initial_energy
        self.packet_size = packet_size
        self.learning_rate = learning_rate
        self.nodes = []
        self.sink = None
        self.cluster_heads = []
        self.time = 0
        self.total_packets_delivered = 0
        self.total_energy_consumed = 0
        self.alive_nodes_history = []
        self.packets_delivered_history = []

        # Setup the network
        self.setup_network()

    def setup_network(self):
        """Initialize the network with nodes"""
        # Create the sink node at the center
        sink_x = self.width / 2
        sink_y = self.height / 2
        self.sink = Node(0, sink_x, sink_y, float('inf'), False, True)
        self.sink.hop_count = 0  # Sink is 0 hops from itself
        self.nodes.append(self.sink)

        # Create regular nodes randomly distributed
        for i in range(1, self.num_nodes):
            x = random.uniform(0, self.width)
            y = random.uniform(0, self.height)
            node = Node(i, x, y, self.initial_energy)
            self.nodes.append(node)

        # Select cluster heads
        self.select_cluster_heads()

        # Establish neighbors based on transmission range
        self.establish_neighbors()

        # Assign nodes to clusters
        self.assign_clusters()

        # Initialize hop counts
        self.initialize_hop_counts()

    def select_cluster_heads(self):
        """Select cluster heads randomly from nodes (excluding sink)"""
        self.cluster_heads = []
        non_sink_nodes = [n for n in self.nodes if not n.is_sink]

        # Clear previous cluster head status
        for node in non_sink_nodes:
            node.is_cluster_head = False

        # Select new cluster heads
        cluster_heads = random.sample(non_sink_nodes, min(self.num_clusters, len(non_sink_nodes)))
        for ch in cluster_heads:
            ch.is_cluster_head = True
            self.cluster_heads.append(ch)

    def establish_neighbors(self):
        """Establish neighbor relationships based on transmission range"""
        for node in self.nodes:
            node.neighbors = []
            for other in self.nodes:
                if node.id != other.id and node.distance_to(other) <= self.transmission_range:
                    node.add_neighbor(other)

    def assign_clusters(self):
        """Assign each node to the nearest cluster head"""
        for node in self.nodes:
            if node.is_sink or node.is_cluster_head:
                continue

            # Find nearest cluster head
            nearest_ch = min(self.cluster_heads, key=lambda ch: node.distance_to(ch))
            node.cluster_head = nearest_ch

    def initialize_hop_counts(self):
        """Initialize hop counts from each node to the sink using BFS"""
        # Reset hop counts
        for node in self.nodes:
            if not node.is_sink:
                node.hop_count = float('inf')

        # BFS from sink
        queue = [self.sink]
        visited = {self.sink.id}

        while queue:
            current = queue.pop(0)
            for neighbor in current.neighbors:
                if neighbor.id not in visited and neighbor.alive:
                    neighbor.hop_count = current.hop_count + 1
                    visited.add(neighbor.id)
                    queue.append(neighbor)

    def calculate_reward(self, current_node, next_node):
        """Calculate the reward for routing from current_node to next_node"""
        if not next_node.alive:
            return float('-inf')  # Can't route through dead nodes

        distance = current_node.distance_to(next_node)
        if distance > self.transmission_range:
            return float('-inf')  # Can't exceed transmission range

        # Calculate normalized distance
        max_distance = max(self.width, self.height)
        normalized_distance = distance / max_distance

        # Calculate n parameter (distance factor)
        n = (normalized_distance * (DFR_MAX - DFR_MIN)) + DFR_MIN

        # Calculate reward based on energy, distance, and hop count
        if next_node.hop_count == float('inf'):
            hop_count_term = 0.001  # Avoid division by zero
        else:
            hop_count_term = next_node.hop_count

        reward = next_node.energy / ((distance ** n) * hop_count_term)
        return reward

    def route_packet(self, source_node):
        """Route a packet from source node to sink using Q-learning"""
        if not source_node.alive or source_node.is_sink:
            return False

        current = source_node
        path = [current.id]
        total_energy = 0

        # Temporary energy deduction to track consumption during routing
        temp_energy_deductions = {}

        max_hops = 20  # Prevent infinite loops
        for _ in range(max_hops):
            if current.is_sink:
                # Successfully reached the sink
                break

            # Get next hop based on Q-values
            next_hop = current.get_next_hop()
            if next_hop is None:
                # No valid next hop found
                return False

            # Calculate energy consumed in this transmission
            distance = current.distance_to(next_hop)
            tx_energy = current.transmit_energy(distance, self.packet_size)
            rx_energy = next_hop.receive_energy(self.packet_size)

            # Check if nodes have enough energy
            if current.energy < tx_energy or next_hop.energy < rx_energy:
                return False

            # Track energy consumption
            temp_energy_deductions[current.id] = temp_energy_deductions.get(current.id, 0) + tx_energy
            temp_energy_deductions[next_hop.id] = temp_energy_deductions.get(next_hop.id, 0) + rx_energy
            total_energy += tx_energy + rx_energy

            # Calculate reward and update Q-value
            reward = self.calculate_reward(current, next_hop)

            # Get maximum Q-value for next node's neighbors
            max_q_next = 0
            if next_hop.neighbors:
                max_q_next = max([next_hop.q_values.get(n.id, 0) for n in next_hop.neighbors])

            current.update_q_value(next_hop.id, reward, max_q_next, self.learning_rate)

            # Move to next hop
            current = next_hop
            path.append(current.id)

            if current.is_sink:
                # Successfully reached the sink
                break

        # If we reached the sink, actually deduct the energy
        if current.is_sink:
            for node_id, energy in temp_energy_deductions.items():
                node = next(n for n in self.nodes if n.id == node_id)
                node.consume_energy(energy)
            self.total_energy_consumed += total_energy
            self.total_packets_delivered += 1
            return True

        return False

    def sleep_scheduling(self, sleep_threshold, sleep_duration):
        """Implement sleep scheduling for energy conservation"""
        for node in self.nodes:
            if not node.alive or node.is_sink or node.is_cluster_head:
                continue

            if node.state == "ACTIVE":
                # Increment counter if no data sent
                if not node.should_transmit_data(CHANGE_THRESHOLD):
                    node.sleep_counter += 1
                else:
                    node.sleep_counter = 0

                # If counter exceeds threshold, put node to sleep
                if node.sleep_counter >= sleep_threshold:
                    node.state = "SLEEP"
                    node.sleep_counter = 0

            elif node.state == "SLEEP":
                # Wake up after sleep duration
                node.sleep_counter += 1
                if node.sleep_counter >= sleep_duration:
                    node.state = "ACTIVE"
                    node.sleep_counter = 0

    def rotate_cluster_heads(self):
        """Periodically rotate cluster heads to distribute energy consumption"""
        if self.time % CLUSTER_ROTATION_INTERVAL == 0 and self.time > 0:
            self.select_cluster_heads()
            self.assign_clusters()

    def run_simulation(self, num_rounds):
        """Run the simulation for the specified number of rounds"""
        for round_num in range(num_rounds):
            self.time += 1

            # Rotate cluster heads periodically
            self.rotate_cluster_heads()

            # Apply sleep scheduling
            self.sleep_scheduling(SLEEP_THRESHOLD, SLEEP_DURATION)

            # Each active node senses and potentially sends data
            for node in self.nodes:
                if not node.alive or node.is_sink:
                    continue

                if node.state == "ACTIVE":
                    # Sense data (consume energy)
                    node.consume_energy(node.sense_energy())

                    # Read sensor data
                    node.read_sensor_data()

                    # Check if data should be transmitted
                    if node.should_transmit_data(CHANGE_THRESHOLD):
                        # If node is a cluster head, route directly
                        if node.is_cluster_head:
                            self.route_packet(node)
                        # Otherwise, send to cluster head first
                        elif node.cluster_head and node.cluster_head.alive:
                            # Calculate energy for transmission to cluster head
                            distance = node.distance_to(node.cluster_head)
                            tx_energy = node.transmit_energy(distance, self.packet_size)
                            rx_energy = node.cluster_head.receive_energy(self.packet_size)

                            if node.energy >= tx_energy and node.cluster_head.energy >= rx_energy:
                                node.consume_energy(tx_energy)
                                node.cluster_head.consume_energy(rx_energy)
                                # Now cluster head forwards to sink
                                self.route_packet(node.cluster_head)

            # Update network statistics
            alive_count = sum(1 for node in self.nodes if node.alive and not node.is_sink)
            self.alive_nodes_history.append(alive_count)
            self.packets_delivered_history.append(self.total_packets_delivered)

            # End simulation if all nodes are dead
            if alive_count == 0:
                print(f"All nodes died at round {round_num}")
                break

        return {
            'alive_nodes': self.alive_nodes_history,
            'packets_delivered': self.packets_delivered_history,
            'total_energy': self.total_energy_consumed,
            'time': self.time
        }

    def plot_results(self):
        """Plot simulation results"""
        plt.figure(figsize=(15, 10))

        # Plot network topology
        plt.subplot(2, 2, 1)
        for node in self.nodes:
            if node.is_sink:
                plt.scatter(node.x, node.y, color='red', s=100, marker='s', label='Sink')
            elif node.is_cluster_head:
                plt.scatter(node.x, node.y, color='green', s=50, marker='^', label='Cluster Head')
            elif node.alive:
                plt.scatter(node.x, node.y, color='blue', s=20, label='Node')
            else:
                plt.scatter(node.x, node.y, color='gray', s=20, alpha=0.5, label='Dead Node')

        plt.title('Network Topology')
        plt.xlabel('X-coordinate')
        plt.ylabel('Y-coordinate')
        handles, labels = plt.gca().get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        plt.legend(by_label.values(), by_label.keys())

        # Plot alive nodes over time
        plt.subplot(2, 2, 2)
        plt.plot(self.alive_nodes_history)
        plt.title('Number of Alive Nodes over Time')
        plt.xlabel('Round')
        plt.ylabel('Number of Alive Nodes')

        # Plot packets delivered over time
        plt.subplot(2, 2, 3)
        plt.plot(self.packets_delivered_history)
        plt.title('Cumulative Packets Delivered over Time')
        plt.xlabel('Round')
        plt.ylabel('Number of Packets')

        # Plot energy consumption distribution
        plt.subplot(2, 2, 4)
        energy_levels = [n.energy/n.initial_energy*100 if not n.is_sink else 0 for n in self.nodes]
        plt.hist(energy_levels, bins=10, range=(0, 100))
        plt.title('Energy Level Distribution')
        plt.xlabel('Remaining Energy (%)')
        plt.ylabel('Number of Nodes')

        plt.tight_layout()
        plt.show()

In [ ]:
TRANSMISSION_RANGE = 100  # meters
E_ELEC = 50e-9  # Energy for electronics (J/bit)
E_AMP = 100e-12  # Energy for amplifier (J/bit/m^2)
E_SENSING = 0.1e-3  # Energy for sensing (J)
E_PROC = 0.2e-3  # Energy for processing (J)
PACKET_SIZE = 500  # bits
DISCOUNT_FACTOR = 0.9  # Q-learning discount factor
CHANGE_THRESHOLD = 2.0  # Threshold for data transmission
SLEEP_THRESHOLD = 5  # Rounds before sleeping
SLEEP_DURATION = 3  # Rounds to sleep
CLUSTER_ROTATION_INTERVAL = 20  # Rounds before rotating cluster heads
DFR_MIN = 2.0  # Distance factor range minimum
DFR_MAX = 4.0  # Distance factor range maximum

In [ ]:
# Simulation parameters
WIDTH = 500  # meters
HEIGHT = 500  # meters
NUM_NODES = 100
NUM_CLUSTERS = 5
INITIAL_ENERGY = 0.5  # Joules
LEARNING_RATE = 0.1
NUM_ROUNDS = 1000

results = {}

In [ ]:
def compare_protocols():
    """Compare RLBEEP with other protocols"""
    # Initialize results dictionary
    results = {}

    # Run RLBEEP simulation
    print("Running RLBEEP simulation...")
    wsn_rlbeep = WSN(WIDTH, HEIGHT, NUM_NODES, NUM_CLUSTERS, TRANSMISSION_RANGE,
                     INITIAL_ENERGY, PACKET_SIZE, LEARNING_RATE)
    results['RLBEEP'] = wsn_rlbeep.run_simulation(NUM_ROUNDS)

    # Run simplified RLBR (without sleep scheduling and data restriction)
    print("Running RLBR simulation...")
    global SLEEP_THRESHOLD, CHANGE_THRESHOLD
    original_sleep_threshold = SLEEP_THRESHOLD
    original_change_threshold = CHANGE_THRESHOLD

    # Disable sleep scheduling and data restriction for RLBR
    SLEEP_THRESHOLD = float('inf')  # Never sleep
    CHANGE_THRESHOLD = 0  # Always transmit

    wsn_rlbr = WSN(WIDTH, HEIGHT, NUM_NODES, NUM_CLUSTERS, TRANSMISSION_RANGE,
                  INITIAL_ENERGY, PACKET_SIZE, LEARNING_RATE)
    results['RLBR'] = wsn_rlbr.run_simulation(NUM_ROUNDS)

    # Run simplified DADF (with data fusion but modified Q-learning)
    print("Running DADF simulation...")
    SLEEP_THRESHOLD = original_sleep_threshold
    CHANGE_THRESHOLD = original_change_threshold

    # For DADF, we'll use a simplified reward function
    original_calculate_reward = WSN.calculate_reward

    def simplified_reward(self, current_node, next_node):
        if not next_node.alive:
            return float('-inf')

        distance = current_node.distance_to(next_node)
        if distance > self.transmission_range:
            return float('-inf')

        # Simplified reward based on energy and hop count only
        if next_node.hop_count == float('inf'):
            hop_count_term = 0.001
        else:
            hop_count_term = next_node.hop_count

        reward = next_node.energy / hop_count_term
        return reward

    # Replace the reward function
    WSN.calculate_reward = simplified_reward

    wsn_dadf = WSN(WIDTH, HEIGHT, NUM_NODES, NUM_CLUSTERS, TRANSMISSION_RANGE,
                  INITIAL_ENERGY, PACKET_SIZE, LEARNING_RATE)
    results['DADF'] = wsn_dadf.run_simulation(NUM_ROUNDS)

    # Restore original function and parameters
    WSN.calculate_reward = original_calculate_reward
    SLEEP_THRESHOLD = original_sleep_threshold
    CHANGE_THRESHOLD = original_change_threshold

    # Plot comparison results
    plt.figure(figsize=(15, 10))

    # Plot alive nodes comparison
    plt.subplot(2, 1, 1)
    for protocol, data in results.items():
        plt.plot(data['alive_nodes'], label=protocol)
    plt.title('Number of Alive Nodes over Time')
    plt.xlabel('Round')
    plt.ylabel('Number of Alive Nodes')
    plt.legend()

    # Plot packets delivered comparison
    plt.subplot(2, 1, 2)
    for protocol, data in results.items():
        plt.plot(data['packets_delivered'], label=protocol)
    plt.title('Cumulative Packets Delivered over Time')
    plt.xlabel('Round')
    plt.ylabel('Number of Packets')
    plt.legend()

    plt.tight_layout()
    plt.show()

    # Print numerical results
    print("\nProtocol Comparison Results:")
    print("-" * 60)
    print(f"{'Protocol':<10} {'First Node Death':<20} {'Network Lifetime':<20} {'Total Packets':<15}")
    print("-" * 60)

    for protocol, data in results.items():
        # Find round when first node died (when alive_nodes decreases)
        alive = data['alive_nodes']
        first_death = next((i for i in range(1, len(alive)) if alive[i] < alive[i-1]), len(alive))

        # Network lifetime (last round with at least one node alive)
        lifetime = len(alive)

        # Total packets delivered
        packets = data['packets_delivered'][-1]

        print(f"{protocol:<10} {first_death:<20} {lifetime:<20} {packets:<15}")

return results

IndentationError: unexpected indent (<ipython-input-36-77f25b9b00c7>, line 106)

In [ ]:
results = compare_protocols()

# Create a new WSN instance for detailed analysis
wsn = WSN(WIDTH, HEIGHT, NUM_NODES, NUM_CLUSTERS, TRANSMISSION_RANGE,
          INITIAL_ENERGY, PACKET_SIZE, LEARNING_RATE)

# Run simulation
wsn.run_simulation(NUM_ROUNDS)

# Plot results
wsn.plot_results()

Running RLBEEP simulation...


ZeroDivisionError: float division by zero